<a href="https://colab.research.google.com/github/ha2ardbunny/HelloGitHub/blob/master/DS_R1_Distill_Llama_8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install unsloth
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [2]:
from google.colab import userdata
from huggingface_hub import login
login(token=userdata.get('DS_TOKEN'))

In [3]:
from unsloth import FastLanguageModel

max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = userdata.get('DS_TOKEN')
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.50.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down-proj",
        ],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,

)

Unsloth: You added custom modules, but Unsloth hasn't optimized for this.
Beware - your finetuning might be noticeably slower!


Not an error, but Unsloth cannot patch MLP layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2025.3.19 patched 32 layers with 32 QKV layers, 32 O layers and 0 MLP layers.


In [5]:
train_prompt_style = """Below is an instruction describing a task, followed by an input.
Write an appropriate response.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [6]:
EOS_TOKEN = tokenizer.eos_token

In [8]:
# Upload dataset
from google.colab import files
uploaded = files.upload()  # Upload "dataset_FT.jsonl"


Saving dataset_FT.jsonl to dataset_FT.jsonl


In [9]:
import os
print(os.listdir("/content/"))


['.config', 'unsloth_compiled_cache', 'dataset_FT.jsonl', 'huggingface_tokenizers_cache', 'sample_data']


In [10]:
from datasets import load_dataset

# Load dataset
dataset = load_dataset("json", data_files="/content/dataset_FT.jsonl", split="train")


# Formatting function to create text prompts
def formatting_prompts_func(examples):
    texts = []

    for message_list in examples["messages"]:
        user_message = next((msg["content"] for msg in message_list if msg["role"] == "user"), "")
        assistant_message = next((msg["content"] for msg in message_list if msg["role"] == "assistant"), "")

        # Format in the same style as prompt template
        formatted_text = f"""### Instruction:
{user_message}

### Response:
{assistant_message}"""

        texts.append(formatted_text)

    return {"text": texts}

# Apply formatting
dataset = dataset.map(formatting_prompts_func, batched=True)

# Verify new format
print(dataset[0])

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

{'messages': [{'role': 'user', 'content': 'Generate a summary for the event based on the following details:\n\nEvent Name: InvestKL 2024 Planning & Industry Visit\nDate: 09 Jan - 12 Jan 2024\nVenue: Sheraton Hotel,Kuching\nTotal Days: 4 Days\nDelegate Days: 128\nFormat: Physical\nProject Manager: Gloria Pahang\n\nSummary:'}, {'role': 'assistant', 'content': 'The InvestKL 2024 Planning & Industry Visit was successfully held from 09 Jan - 12 Jan 2024 at Sheraton Hotel,Kuching. This physical event lasted for 4 days and gathered approximately 128 delegates. The event was managed by Gloria Pahang, ensuring a smooth execution and a great experience for all attendees.'}], 'text': '### Instruction:\nGenerate a summary for the event based on the following details:\n\nEvent Name: InvestKL 2024 Planning & Industry Visit\nDate: 09 Jan - 12 Jan 2024\nVenue: Sheraton Hotel,Kuching\nTotal Days: 4 Days\nDelegate Days: 128\nFormat: Physical\nProject Manager: Gloria Pahang\n\nSummary:\n\n### Response:\n

In [11]:
# Function to format dataset
def formatting_prompts_func(examples):
    texts = []

    for message_list in examples["messages"]:
        user_message = next((msg["content"] for msg in message_list if msg["role"] == "user"), "")
        assistant_message = next((msg["content"] for msg in message_list if msg["role"] == "assistant"), "")

        # Identify if the prompt is for summarization or general AI
        if any(keyword in user_message.lower() for keyword in ["event", "conference", "summarize", "summarization"]):
            instruction = "Summarize the event details accurately."
            formatted_text = train_prompt_style.format(instruction, user_message, assistant_message)
        else:
            instruction = "Answer the following question in a clear and professional manner."
            formatted_text = train_prompt_style.format(instruction, user_message, assistant_message)

        texts.append(formatted_text + EOS_TOKEN)

    return {"text": texts}

In [12]:
# Apply formatting
dataset = dataset.map(formatting_prompts_func, batched=True)

# Check first formatted example
print(dataset["text"][0])


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Below is an instruction describing a task, followed by an input.
Write an appropriate response.

### Instruction:
Summarize the event details accurately.

### Input:
Generate a summary for the event based on the following details:

Event Name: InvestKL 2024 Planning & Industry Visit
Date: 09 Jan - 12 Jan 2024
Venue: Sheraton Hotel,Kuching
Total Days: 4 Days
Delegate Days: 128
Format: Physical
Project Manager: Gloria Pahang

Summary:

### Response:
The InvestKL 2024 Planning & Industry Visit was successfully held from 09 Jan - 12 Jan 2024 at Sheraton Hotel,Kuching. This physical event lasted for 4 days and gathered approximately 128 delegates. The event was managed by Gloria Pahang, ensuring a smooth execution and a great experience for all attendees.<｜end▁of▁sentence｜>


In [13]:
# Fine-tuning setup
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,

    args=TrainingArguments(
        per_device_train_batch_size=8,  # Increase batch size for stability
        gradient_accumulation_steps=4,  # Reduce for faster training
        warmup_steps=5,  # Reduce warmup since dataset is small
        num_train_epochs=10,  # Instead of steps, train for multiple epochs
        learning_rate=5e-5,  # Lower learning rate for small dataset
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=2,  # More frequent logging
        optim="adamw_8bit",  # Efficient optimizer
        weight_decay=0.01,
        lr_scheduler_type="linear",  # Linear decay works better for small datasets
        seed=3407,
        output_dir="output",
        report_to="none",
    ),
)


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/100 [00:00<?, ? examples/s]

In [14]:
# Train the model
trainer_stats = trainer.train()


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 100 | Num Epochs = 10 | Total steps = 30
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 4 x 1) = 32
 "-____-"     Trainable parameters = 32,505,856/8,000,000,000 (0.41% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
2,3.021800
4,2.949100
6,2.817700
8,2.850900
10,2.410200
12,2.297600
14,2.160600
16,1.779200
18,1.798900
20,1.590800


In [15]:
# Test the fine-tuned model
def generate_response(question):
    if any(keyword in question.lower() for keyword in ["event", "conference", "summarize", "summarization"]):
        instruction = "Summarize the event details accurately."
        formatted_input = train_prompt_style.format(instruction, question, "")
    else:
        instruction = "Answer the following question professionally."
        formatted_input = train_prompt_style.format(instruction, question, "")

    inputs = tokenizer(formatted_input, return_tensors="pt").to("cuda")

    outputs = model.generate(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_new_tokens=256,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
    )

    response = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return response[0]

In [19]:
# Test cases
print(generate_response("Hi, how are you today?"))
print(generate_response("may i know who are you"))
print(generate_response("tell me some jokes."))
print(generate_response("List out definition of all Newton's Law"))
print(generate_response("Tell me about the half-life of uranium-235"))
print(generate_response("Summarize Engineering Conference (EnCon) 2024"))
print(generate_response("List the event held in kuching"))
print(generate_response("Donny Tan involve in which event"))
print(generate_response("Based on this detail, generate the summary.Event Name: The International Conference on Higher Education Learning and Teaching 2024\nDate: 24 Nov - 25 Nov 2023\nVenue: Curtin University Malaysia,Miri\nTotal Days: 2 Days\nDelegate Days: 400\nFormat: Physical\nProject Manager: Donny Tan"))

Below is an instruction describing a task, followed by an input.
Write an appropriate response.

### Instruction:
Answer the following question professionally.

### Input:
Hi, how are you today?

### Response:
Hi! I'm just a computer program, so I don't have feelings, but thanks for asking! How can I assist you today?
Below is an instruction describing a task, followed by an input.
Write an appropriate response.

### Instruction:
Answer the following question professionally.

### Input:
may i know who are you

### Response:
Certainly! How can I assist you today?
Below is an instruction describing a task, followed by an input.
Write an appropriate response.

### Instruction:
Answer the following question professionally.

### Input:
tell me some jokes.

### Response:
Sure! Here's a light-hearted joke: Why don't skeletons fight each other? They don't have the guts!
Below is an instruction describing a task, followed by an input.
Write an appropriate response.

### Instruction:
Answer the 